In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pressao_vapor_perry = pd.read_csv("tabelas/pressao_vapor.csv")
lista_nomes = pressao_vapor_perry['Name'].tolist()

## Pressão de vapor

In [3]:

pressao_vapor_perry = pd.read_csv("tabelas/pressao_vapor.csv")

def calc_pressao_vapor(compound: str, T_user: float):
    row_compound = pressao_vapor_perry[pressao_vapor_perry['Name'] == compound]
    C1 = row_compound['C1'].iloc[0]
    C2 = row_compound['C2'].iloc[0]
    C3 = row_compound['C3'].iloc[0]
    C4 = row_compound['C4'].iloc[0]
    C5 = row_compound['C5'].iloc[0]
    Tmin =row_compound['Tmin'].iloc[0]
    Tmax =row_compound['Tmax'].iloc[0]
    T_range = np.linspace(Tmin, Tmax, 2000)
    # Calculo da pressão de vapor de acordo com a temperatura especifica
    pressao_user = 1e-6*np.exp(C1 + C2/T_user + C3*np.log(T_user) + C4*(T_user**C5))
    pressao_vapor = []
    for T in T_range:
        pressao_vapor.append(1e-6*np.exp(C1 + C2/T + C3*np.log(T) + C4*(T**C5)))
    return pressao_user, T_range, pressao_vapor


## Densidade e volume molar

In [51]:
densidade_molar_perry = pd.read_csv("tabelas/densidade_molar.csv")

def calc_density(compound: str, T_user: float):
    row_compound = densidade_molar_perry[densidade_molar_perry['Name'] == compound]
    C1 = row_compound['C1'].iloc[0]
    C2 = row_compound['C2'].iloc[0]
    C3 = row_compound['C3'].iloc[0]
    C4 = row_compound['C4'].iloc[0]
    Tmin =row_compound['Tmin'].iloc[0]
    Tmax =row_compound['Tmax'].iloc[0]
    T_range = np.linspace(Tmin, Tmax, 2000)
    # Calculo da densidade e volume molar de acordo com a temperatura especifica
    densidade_user = C1 + C2*T_user + C3*(T_user**2) + C4*(T_user**3)
    volume_user = 1/densidade_user
    # Calculo da densidade e volume molar de acordo com a temperatura tabelada
    densidade_molar = []
    volume_molar = []
    for T in T_range:
        if compound == 'Water' or compound == 'o-Terphenyl':
            densidade_molar.append(C1 + C2*T + C3*(T**2) + C4*(T**3)) #mol/dm³
            volume_molar.append(1/(C1 + C2*T + C3*(T**2) + C4*(T**3)))# dm³/mol
        else:
            densidade_molar.append(C1/(C2**(1+(1-T/C3)**C4)))#mol/dm³
            volume_molar.append(1/(C1/(C2**(1+(1-T/C3)**C4))))# dm³/mol
    return densidade_user, volume_user, T_range, densidade_molar, volume_molar


Entalpia de vaporização $\Delta H_{vap}$

In [50]:
entalpia_perry = pd.read_csv("tabelas/entalpia_vap.csv")
criticos_perry = pd.read_csv("tabelas/criticos.csv")

def calc_entalpia_vap(compound: str, T_user: float):
    row_compound = entalpia_perry[entalpia_perry['Name'] == compound]
    row_compound_criticos = criticos_perry[criticos_perry['Name'] == compound]
    Tc = float(row_compound_criticos['Tc'].iloc[0])
    C1 = float(row_compound['C1'].iloc[0])
    C2 = float(row_compound['C2'].iloc[0])
    C3 = float(row_compound['C3'].iloc[0])
    C4 = float(row_compound['C4'].iloc[0])
    Tmin =float(row_compound['Tmin'].iloc[0])
    Tmax =float(row_compound['Tmax'].iloc[0])
    T_range = np.linspace(Tmin, Tmax, 2000)
    # Calcular entalpia com a temperatura específica
    entalpia_user = C1/1000*(1-T_user/Tc)**(C2+C3*T_user/Tc+C4*(T_user/Tc)**2) #J/mol
    # Calcular entalpia em função da temperatura tabelada
    entalpia_vap = []
    for T in T_range:
        entalpia_vap.append(C1/1000*(1-T/Tc)**(C2+C3*T/Tc+C4*(T/Tc)**2)) #J/mol
    return entalpia_user, T_range, entalpia_vap

Hildebrand $\delta$

$\delta = \sqrt{\frac{\Delta H_v-RT}{V}}$

In [15]:
entalpia_perry = pd.read_csv("tabelas/entalpia_vap.csv")
densidade_perry = pd.read_csv("tabelas/densidade_molar.csv")


Interseção entre os arrays: [273.16]


In [32]:
def intersecao_temps(array1, array2, tolerance=0.1):
    # Convertendo os arrays em conjuntos (sets)
    set1 = set(array1)
    set2 = set(array2)

    # Criando um conjunto para armazenar a interseção
    intersecao = set()

    # Percorrendo os elementos do primeiro array
    for num1 in set1:
        # Verificando se o elemento está presente no segundo array dentro da tolerância
        for num2 in set2:
            if abs(num1 - num2) < tolerance:
                # Adicionando o elemento à interseção
                intersecao.add(num1)
                break  # Para evitar adicionar o mesmo elemento mais de uma vez

    # Convertendo a interseção de volta para uma lista, se necessário
    intersecao_lista = list(intersecao)

    return intersecao_lista

# RETOMAR DAQUI. VER O VALOR ERRADO DOS PARÂMETROS

In [62]:

def calc_hildebrand(compound: str,T_user: float):
    entalpia_user,t_range_entalpia,_ = calc_entalpia_vap(compound, T_user)
    _,volume_user,t_range_volume,_,_ = calc_density(compound, T_user)
    R = 8.314462618 #J/mol/K
    # Calcular Hildebrand com a temperatura específica
    hildebrand_user = 1/1000*np.sqrt(1/1000000*(entalpia_user-R*T_user)/volume_user)
    # Calcular Hildebrand nas temperaturas de interseção
    T_range = intersecao_temps(t_range_volume, t_range_entalpia)
    hildebrand = []

    for T in T_range:
        entalpia,_,_ = calc_entalpia_vap(compound, T)
        _,volume,_,_,_ = calc_density(compound, T)
        hildebrand.append(1/1000*np.sqrt(1/1000000*(entalpia-R*T)/volume))
    return hildebrand_user, hildebrand, T_range

calc_hildebrand('Hexane', 273.15+25)

C:\Users\willi\AppData\Local\Temp\ipykernel_26388\2995615117.py:14: RuntimeWarning: invalid value encountered in sqrt
  hildebrand.append(1/1000*np.sqrt(1/1000000*(entalpia-R*T)/volume))


(40.66581963903047,
 [25.701284428067584,
  25.83609195127904,
  25.813633815375628,
  25.9034427886418,
  26.217275736058834,
  26.306798580898608,
  25.85854616202808,
  25.92588519478278,
  26.440961902525906,
  26.574979132796578,
  26.708849189017062,
  26.842570981307244,
  26.127688756862028,
  26.17249024349952,
  27.065108176498608,
  27.24283576632795,
  27.3759534575736,
  26.97614341252134,
  27.220634608595308,
  27.35377782738598,
  28.41329650293089,
  27.4867674106085,
  27.398124814891382,
  27.64172623355751,
  28.89546063446665,
  29.048407494204415,
  29.179322016287813,
  29.24471559237351,
  29.266503977844113,
  29.288287617066523,
  29.310066504248756,
  29.33184063359183,
  29.37537459552959,
  29.397134416491234,
  29.418889456347664,
  29.44063970926474,
  29.46238516940124,
  29.50586168793234,
  29.52759273460903,
  29.636175607104235,
  29.766315081330017,
  29.896278829547313,
  30.047679382096945,
  30.177257950651367,
  30.3066566735618,
  30.4143506076